In [96]:
import os
import re
import pandas as pd
from docx import Document
from lxml import etree

os.chdir('C:/Workspace/R-pharm/work/abbreviation_app')

### Load abb_dict_review and separate to 2 files

In [105]:
BASE_DIR = r"C:\Workspace\R-pharm\work\abbreviation_app\data"
df = pd.read_csv(os.path.join(BASE_DIR, "abb_dict_review.csv"))
df['abbreviation'] = df['abbreviation'].str.strip()
grouped = df.groupby('abbreviation')['description'].nunique().reset_index()

single_desc_abbs = grouped[grouped['description'] == 1]['abbreviation']
multiple_desc_abbs = grouped[grouped['description'] > 1]['abbreviation']

df_single = df[df['abbreviation'].isin(single_desc_abbs)]
df_multiple = df[df['abbreviation'].isin(multiple_desc_abbs)]

df_single.to_csv(os.path.join(BASE_DIR, "single_description_abbs.csv"), index=False, encoding='utf-8-sig')
df_multiple.to_csv(os.path.join(BASE_DIR, "multiple_description_abbs.csv"), index=False, encoding='utf-8-sig')

### Check reviewed multiple_description_abbs.csv

In [51]:
BASE_DIR = r"C:\Workspace\R-pharm\work\abbreviation_app\data"
df = pd.read_csv(os.path.join(BASE_DIR, "multiple_description_abbs_reviewed.csv"))
df['abbreviation'] = df['abbreviation'].str.strip()
grouped = df.groupby('abbreviation')['description'].nunique().reset_index()

multiple_desc_abbs = grouped[grouped['description'] > 1]['abbreviation']
df_multiple = df[df['abbreviation'].isin(multiple_desc_abbs)]

for abb, group in df_multiple.groupby('abbreviation'):
    print(f"Abbreviation: {abb}")
    print("Descriptions:")
    print("\n".join(group['description'].tolist()))
    print("-" * 40)

Abbreviation: ADA
Descriptions:
Adalimumab (Адалимумаб)
American Diabetes Association (Американская диабетологическая ассоциация)
----------------------------------------
Abbreviation: BCR
Descriptions:
Breakpoint Cluster Region (область кластерного разрыва): Участок ДНК, кодирующий одноименный белок
B Cell Receptor (В-клеточный рецептор)
----------------------------------------
Abbreviation: ER
Descriptions:
Estrogen Receptor (рецептор эстрогенов)
Exposure Ratio (коэффициент экспозиции/коэффициент воздействия)
----------------------------------------
Abbreviation: F
Descriptions:
Bioavailability (биодоступность)
Females (самки)
----------------------------------------
Abbreviation: M
Descriptions:
Mean (среднее арифметическое)
Males (самцы)
----------------------------------------
Abbreviation: MF
Descriptions:
Marketed Form (вариант препарата, реализуемый на рынке)
Matrix Factor (матричный фактор)
----------------------------------------
Abbreviation: Pgp
Descriptions:
P-гликопротеин

### Combine 2 reviewed files back/check different descriptions for the same abb

In [49]:
pd.set_option('display.max_rows', None)
def clean_and_sort_abbreviations(df):
    """
    Cleans, deduplicates, and sorts the abbreviation DataFrame.
    - Capitalizes and strips the description.
    - Removes duplicates.
    - Sorts by abbreviation and description.
    """
    df['description'] = df['description'].str.strip().apply(
        lambda x: x[0].upper() + x[1:] if x else x
    )
    return (df
            .drop_duplicates()
            .sort_values(by=['abbreviation', 'description'])
            .reset_index(drop=True)
           )


BASE_DIR = r"C:\Workspace\R-pharm\work\abbreviation_app\data"
# df1 = pd.read_csv(os.path.join(BASE_DIR, "single_description_abbs_reviewed.csv"))
# df2 = pd.read_csv(os.path.join(BASE_DIR, "multiple_description_abbs_reviewed.csv"))

# combined_abbs = pd.concat([df1, df2], ignore_index=True)
# combined_abbs['abbreviation'] = combined_abbs['abbreviation'].str.strip()
# combined_abbs['description'] = combined_abbs['description'].str.capitalize().str.strip()
# combined_abbs = clean_and_sort_abbreviations(combined_abbs)

# combined_abbs.to_csv(os.path.join(BASE_DIR, "abb_dict.csv"), index=False, encoding='utf-8-sig')
combined_abbs = pd.read_csv(os.path.join(BASE_DIR, "abb_dict.csv"))
inconsistent = (combined_abbs.groupby('abbreviation')['description']
                .nunique()
                .reset_index()
                .query('description > 1')
                )

count_inconsistent = inconsistent['abbreviation'].nunique()
inconsistent_abbs = combined_abbs[combined_abbs['abbreviation'].isin(inconsistent['abbreviation'])]

print(f"\n[INFO] Abbreviations with more than one unique description: {count_inconsistent}")
inconsistent_abbs


[INFO] Abbreviations with more than one unique description: 33


,abbreviation,description
17,ADA,Adalimumab (адалимумаб)
18,ADA,American diabetes association (американская диабетологическая ассоциация)
81,BCR,B cell receptor (в-клеточный рецептор)
82,BCR,"Breakpoint cluster region (область кластерного разрыва): участок днк, кодирующий одноименный белок"
232,ER,Estrogen receptor (рецептор эстрогенов)
233,ER,Exposure ratio (коэффициент экспозиции/коэффициент воздействия)
245,F,Bioavailability (биодоступность)
246,F,Females (самки)
394,M,Males (самцы)
395,M,Mean (среднее арифметическое)


### Check same descriptions for different abbs

In [52]:
BASE_DIR = r"C:\Workspace\R-pharm\work\abbreviation_app\data"
combined_abbs = pd.read_csv(os.path.join(BASE_DIR, "abb_dict.csv"))
duplicates = combined_abbs.groupby('description').filter(lambda x: len(x) > 1)
duplicates.sort_values(by='description')

,abbreviation,description
671,V/F,"Apparent Volume of Distribution (кажущийся объем распределения): Гипотетический объем, в котором лекарственное вещество распределяется после внесосудистого введения"
670,Vd/F,"Apparent Volume of Distribution (кажущийся объем распределения): Гипотетический объем, в котором лекарственное вещество распределяется после внесосудистого введения"
173,Cl/F,"Apparent clearance (кажущийся клиренс): общий клиренс лекарственного вещества после внесосудистого введения, учитывающий биодоступность (f)"
137,CL/F,"Apparent clearance (кажущийся клиренс): общий клиренс лекарственного вещества после внесосудистого введения, учитывающий биодоступность (f)"
48,AUC(0-24),Area under the curve from 0 to 24 hours (площадь под кривой «концентрация-время» от нуля до 24 часов): отражает экспозицию препарата в течение первых суток
54,AUC0-24,Area under the curve from 0 to 24 hours (площадь под кривой «концентрация-время» от нуля до 24 часов): отражает экспозицию препарата в течение первых суток
58,AUC0–24,Area under the curve from 0 to 24 hours (площадь под кривой «концентрация-время» от нуля до 24 часов): отражает экспозицию препарата в течение первых суток
60,AUC24h,Area under the curve from 0 to 24 hours (площадь под кривой «концентрация-время» от нуля до 24 часов): отражает экспозицию препарата в течение первых суток
52,AUC(0-∞),"Area under the curve from 0 to infinity (площадь под кривой от нуля до бесконечности): отражает общую экспозицию препарата, включая экстраполяцию за пределы последнего отбора крови с измеряемой концентрацией"
57,AUC0-∞,"Area under the curve from 0 to infinity (площадь под кривой от нуля до бесконечности): отражает общую экспозицию препарата, включая экстраполяцию за пределы последнего отбора крови с измеряемой концентрацией"


### Check for mixed language

In [53]:
BASE_DIR = r"C:\Workspace\R-pharm\work\abbreviation_app\data"
df = pd.read_csv(os.path.join(BASE_DIR, "abb_dict.csv"))

mixed_pattern = re.compile(r'(?=.*[A-Za-z])(?=.*[А-Яа-яЁё])')

df['abbreviation'][df['abbreviation'].str.contains(mixed_pattern)]

55              AUC0-3 ч
112       Bridge-терапия
186                  CРБ
187         DAS28-4(СОЭ)
714             n-3 ПНЖК
779             Анти-HCV
780                 АпоB
781                 АпоE
810                  ВГB
811                  ВГC
812                  ВГB
814                  ВГC
939               ИМбпST
940                ИМпST
960                  Кel
1099             ОКСбпST
1100              ОКСпST
1279    УВЭЖХ-qTOF/МС/МС
1367           анти-VEGF
Name: abbreviation, dtype: object

### Check for misstyped mixed language abbs

In [60]:
def check_and_correct_misstyped_abb(abb, abb_dict):
    """
    Подбирает правильную форму аббревиатуры (смешанная кириллица/латиница),
    сверяясь с уже имеющимся словарём (abb_dict).
    """

    cyr2lat = {
        'А': 'A', 'В': 'B', 'С': 'C', 'Е': 'E', 'Н': 'H',
        'К': 'K', 'М': 'M', 'О': 'O', 'Р': 'P', 'Т': 'T',
        'У': 'Y', 'Х': 'X', 'Г': 'G'  # добавили 'Г': 'G' при необходимости
    }
    lat2cyr = {v: k for k, v in cyr2lat.items()}

    # Генерация ВСЕХ возможных смешанных форм
    possible_forms = generate_all_mixed_forms(abb, cyr2lat, lat2cyr)

    # Ищем среди полученных форм те, что уже есть в abb_dict
    matches = []
    for form in possible_forms:
        row = abb_dict[abb_dict['abbreviation'] == form]
        if not row.empty:
            # Если совпало, добавляем (форма, описание) в список
            desc = row['description'].values[0]
            matches.append((form, desc))

    # Если ничего не нашли, возвращаем исходную
    if not matches:
        return abb

    # Удалим дубликаты (если в словаре несколько одинаковых описаний)
    matches = list(dict.fromkeys(matches))

    # Если ровно одна форма нашлась
    if len(matches) == 1:
        corrected_abb, description = matches[0]
        highlighted = highlight_mixed_characters(abb, cyr2lat, lat2cyr)
        print(f"[WARNING] '{highlighted}' appears mistyped.")
        print(f"Did you mean '{corrected_abb}'? - {description}")

        choice = input(f"Use '{corrected_abb}' instead of '{abb}'? (y/n): ").strip().lower()
        return corrected_abb if choice == 'y' else abb

    # Если несколько вариантов
    highlighted = highlight_mixed_characters(abb, cyr2lat, lat2cyr)
    print(f"[WARNING] '{highlighted}' appears mistyped. Possible corrections:")
    for i, (corr, desc) in enumerate(matches, start=1):
        print(f" {i}. {corr} - {desc}")

    choice = input(f"Select the correct abbreviation for '{abb}' (1-{len(matches)}) or Enter to skip: ").strip()
    if choice.isdigit():
        idx = int(choice)
        if 1 <= idx <= len(matches):
            corrected_abb, _ = matches[idx - 1]
            confirm = input(f"Use '{corrected_abb}' instead of '{abb}'? (y/n): ").strip().lower()
            if confirm == 'y':
                return corrected_abb

    return abb

def generate_all_mixed_forms(abb, cyr2lat, lat2cyr):
    """
    Generates all possible forms of the abbreviation by trying:
      - leaving the character as is,
      - converting from Cyrillic to Latin (if possible),
      - converting from Latin to Cyrillic (if possible).
    Returns a set of all generated combinations.
    """
    results = set()

    def backtrack(i, current):
        if i == len(abb):
            results.add("".join(current))
            return

        ch = abb[i]
        # 1) Leave the character as is
        current.append(ch)
        backtrack(i + 1, current)
        current.pop()

        # 2) Cyrillic -> Latin (if ch is in cyr2lat)
        if ch in cyr2lat:
            current.append(cyr2lat[ch])
            backtrack(i + 1, current)
            current.pop()

        # 3) Latin -> Cyrillic (if ch is in lat2cyr)
        if ch in lat2cyr:
            current.append(lat2cyr[ch])
            backtrack(i + 1, current)
            current.pop()

    backtrack(0, [])
    return results


def highlight_mixed_characters(abb, cyr2lat, lat2cyr):
    """
    Highlights each character by appending (Cyr) or (Lat) if it belongs
    to the respective conversion dictionary.
    """
    def mark(ch):
        if ch in cyr2lat:
            return f"{ch}(Cyr)"
        elif ch in lat2cyr:
            return f"{ch}(Lat)"
        return ch

    return "".join(mark(ch) for ch in abb)


abb_dict = pd.read_csv(os.path.join(BASE_DIR, "abb_dict.csv"))
new_abbs = ['BГВ']
for abb in new_abbs:
    corrected_abb = check_and_correct_misstyped_abb(abb, abb_dict)
    if corrected_abb != abb:
        print(f"[INFO] Using abbreviation: {corrected_abb}")
    else:
        print(f"[INFO] No correction for: {abb}")

[WARNING] 'B(Lat)Г(Cyr)В(Cyr)' appears mistyped.
Did you mean 'ВГB'? - Вирусный гепатит B
[INFO] Using abbreviation: ВГB


### Debug quoted mathes

In [149]:
text = (
        "A1/3, R-111, B-положительный, Ёёк53б Cint: ЖghhпПП, ЖЖЖ-001, "
        "«оценка по шкалам (АБСд) и ABCd», BBB, as shown in «AAA» and «BB12 smth», "
        "not in («XXds»)."
)

text_no_space_quoted = re.compile(r'«\S+»').sub('', text)
text_no_space_quoted

pattern = re.compile(r'\b(«?[A-ZА-ЯЁ]{2,}[a-zа-яёЁ\d-]*[A-Za-zА-Яа-яёЁ\d]*»?)\b')
matches = pattern.findall(text_no_space_quoted)
matches
# ['ЖЖЖ-001', 'АБСд', 'ABCd', 'BBB', 'BB12']

['ЖЖЖ-001', 'АБСд', 'ABCd', 'BBB', 'BB12']

In [194]:
doc_abbs = set()
text = (
        "BCR-ABL, COVID-19, Child–Pugh Cl/F DAS28-4(СОЭ), AUC(0-24), HbsAg AUC24,ss, ATV/c Ёёк53б Cint: ЖghhпПП, ЖЖЖ-001, "
        "«оценка по шкалам (АБСд) и ABCd», as shown in «AAA» and «BB12 smth», "
        "not in («XXds»), (XIX, VIа, где а написано кириллицей, IIB и IIIa, где а – латинская буква, IV, IVb, VII)."
)

text_no_space_quoted = re.compile(r'«\S+»').sub('', text)
roman_pattern = re.compile(r'^[IVXLCDM]+[A-Za-zА-Яа-яёЁ]*$', re.IGNORECASE)

words = text_no_space_quoted.split()
matches = [word for word in words if re.search(r'[A-ZА-ЯЁ].*[A-ZА-ЯЁ]', word)]

for match in matches:
    clean_match = match.strip(',.»«')
    if clean_match.startswith('('):
        clean_match = clean_match[1:]
    if clean_match.endswith(')') and clean_match.count('(') == 0:
        clean_match = clean_match[:-1]
    if (not roman_pattern.match(clean_match)):
        doc_abbs.add(clean_match)

doc_abbs
# ['ЖghhпПП','ЖЖЖ-001', 'АБСд', 'ABCd', 'BBB', 'BB12']

{'ABCd',
 'ATV/c',
 'AUC(0-24)',
 'AUC24,ss',
 'BB12',
 'BCR-ABL',
 'COVID-19',
 'Child–Pugh',
 'Cl/F',
 'DAS28-4(СОЭ)',
 'HbsAg',
 'АБСд',
 'ЖghhпПП',
 'ЖЖЖ-001'}

### Debug stop section

In [120]:
file_path = "data/abb_examples/tmp/DT_VIME_CA10889127_Отчет_о_КИ_БЭ_Тело_отчета_в1_0_15мая2024_финал.docx"
doc = Document(file_path)
skip_sections = [
        "СПИСОК ЛИТЕРАТУРЫ",
        "Список использованной литературы",
        "Список использованных источников"
    ]

for para in doc.paragraphs:
    para_text = para.text.strip()
    
    is_heading = (
        para.style.name.startswith('Heading') or
        'Заголовок' in para.style.name
    )
    is_bold = any(run.bold for run in para.runs if run.text.strip())
    
    if any(t.upper() in para_text.upper() for t in skip_sections):
            print(f"[DEBUG] Detected: {para_text} - Style: {para.style.name} - Is Bold: {is_bold} - Is Heading: {is_heading}")

    #if (is_bold or is_heading) and any(title in para_text.upper() for title in skip_sections):
        #print(para_text)

[DEBUG] Detected: 11.	СПИСОК ЛИТЕРАТУРЫ	580 - Style: toc 1 - Is Bold: False - Is Heading: False
[DEBUG] Detected: СПИСОК ЛИТЕРАТУРЫ - Style: Heading 1 - Is Bold: False - Is Heading: True


In [124]:
abb = "ApoE"
cyr2lat = {
    'А': 'A',  # Cyrillic А -> Latin A
    'В': 'B',  # Cyrillic В -> Latin B
    'С': 'C',  # Cyrillic С -> Latin C
    'Е': 'E',  # Cyrillic Е -> Latin E
    'Н': 'H',  # Cyrillic Н -> Latin H
    'К': 'K',  # Cyrillic К -> Latin K
    'М': 'M',  # Cyrillic М -> Latin M
    'О': 'O',  # Cyrillic О -> Latin O
    'Р': 'P',  # Cyrillic Р -> Latin P
    'Т': 'T',  # Cyrillic Т -> Latin T
    'У': 'Y',  # Cyrillic У -> Latin Y
    'Х': 'X'   # Cyrillic Х -> Latin X
}
lat2cyr = {v: k for k, v in cyr2lat.items()}

is_mixed = any(char in cyr2lat for char in abb) and \
            any(char in lat2cyr for char in abb)
is_mixed


False

In [ ]:
file_path = "C:/Workspace/R-pharm/work/abbreviation_app/data/abb_examples/TL-NLT-c-01_БИ_в2.0_19сен2018_драфт_ОФ.docx"
doc = Document(file_path)

for para in doc.paragraphs:
    para_text = para.text.strip()
    is_bold = any(run.bold for run in para.runs if run.text.strip())
    
    if is_bold and any(t.upper() in para_text.upper() for t in skip_section):
        print(t)

In [196]:
def extract_abbreviations_from_doc(doc):
    """
    Extracts uppercase and mixed-case abbreviations from the document.
    Stops searching at "Список литературы" if the style is Heading1.
    Excludes pure Roman numerals, specific terms, and words in quotes.
    """
    doc_abbs = set()

    exclude_terms = {
        'ПРОТОКОЛ', 
        'КЛИНИЧЕСКОГО', 
        'ИССЛЕДОВАНИЯ'
    }
    roman_pattern = re.compile(r'^[IVXLCDM]+[A-Za-zА-Яа-яёЁ]*$', re.IGNORECASE)
    stop_section = "Список литературы"

    for para in doc.paragraphs:
        if stop_section.lower() in para.text.lower() and para.style.name == 'Heading 1':
            break
        text_no_space_quoted = re.compile(r'«\S+»').sub('', para.text)
        words = text_no_space_quoted.split()
        matches = [word for word in words if re.search(r'[A-ZА-ЯЁ].*[A-ZА-ЯЁ]', word)]
        
        for match in matches:
            clean_match = match.strip(':,.»«')
            if clean_match.startswith('('):
                clean_match = clean_match[1:]
            if clean_match.endswith(')') and clean_match.count('(') == 0:
                clean_match = clean_match[:-1]
            if (
                not roman_pattern.match(clean_match)
                and clean_match not in exclude_terms
                ):
                doc_abbs.add(clean_match)
    return doc_abbs

extract_abbreviations_from_doc(doc)

{'A-Z',
 'ABCd',
 'ABCs',
 'AE',
 'ATХ',
 'AUC(0-24)',
 'AUC0-24',
 'DC01',
 'EMA',
 'FDA',
 'GCP',
 'NCI',
 'NCI-CTC',
 'R-R',
 'RR',
 'А-Я',
 'АТХ',
 'ВАК-123',
 'ВГВ',
 'КИ',
 'СТСАЕ',
 'ФК'}

### Format abbs

In [62]:
ABB_DICT_PATH = "data/abb_dict.csv"
abb_dict = pd.read_csv(ABB_DICT_PATH)
row = abb_dict.iloc[723]
print(row)

abbr = row['abbreviation']
desc = row['description']

# Convert abbreviation (with Greek letters) to uppercase Latin letters
greek_to_latin = {
        'α': 'A', 'β': 'B', 'γ': 'G', 'δ': 'D',
        'ε': 'E', 'ζ': 'Z', 'η': 'H', 'θ': 'TH',
        'ι': 'I', 'κ': 'K', 'λ': 'L', 'μ': 'M',
        'ν': 'N', 'ξ': 'X', 'ο': 'O', 'π': 'P',
        'ρ': 'R', 'σ': 'S', 'τ': 'T', 'υ': 'U',
        'φ': 'PH', 'χ': 'CH', 'ψ': 'PS', 'ω': 'O'
    }
abbr = ''.join(greek_to_latin.get(char, char) for char in abbr).upper()
abbr_letters = ''.join(re.findall(r'[A-Z]', abbr))

def capitalize_by_abbreviation(english_desc, abbr_letters):
    abbr_idx = 0 # position in the abbreviation
    pos = 0      # position in the description
    desc_list = list(english_desc)

    while abbr_idx < len(abbr_letters) and pos < len(desc_list):
        if (desc_list[pos].lower() == abbr_letters[abbr_idx].lower() and
            (pos == 0 or not desc_list[pos - 1].isalpha())):
            desc_list[pos] = desc_list[pos].upper()
            abbr_idx += 1
        pos += 1
    return ''.join(desc_list)

parts = desc.split('(', 1)
english_desc = parts[0].strip().lower()
russian_desc = '(' + parts[1] if len(parts) > 1 else ''
print(english_desc)
print(russian_desc)

capitalize_by_abbreviation(english_desc, abbr_letters)


abbreviation                                               α1-AGP
description     Alpha-1-Acid Glycoprotein (α1-кислый гликопрот...
Name: 723, dtype: object
alpha-1-acid glycoprotein
(α1-кислый гликопротеин)


'Alpha-1-Acid Glycoprotein'

In [69]:
ABB_DICT_PATH = "data/abb_dict.csv"
abb_dict = pd.read_csv(ABB_DICT_PATH)

invalid_rows = abb_dict[
    abb_dict['description'].str.contains(r'[\x00-\x1F\x7F]', na=False)
]

if not invalid_rows.empty:
    print("[ERROR] Invalid descriptions found:\n", invalid_rows)
    raise ValueError("Control characters detected. Please clean the data.")

In [72]:
invalid_rows = abb_dict[
    abb_dict['abbreviation'].str.contains(r'[\x00-\x1F\x7F]', na=False)
]

if not invalid_rows.empty:
    print("[ERROR] Invalid abbreviation found:\n", invalid_rows)
    raise ValueError("Control characters detected. Please clean the data.")

### Exclude words

In [73]:
exclude_terms = {
        'ПРОТОКОЛ', 'КЛИНИЧЕСКОГО', 'ИССЛЕДОВАНИЯ',
        'ПЕРИОД', 'ИССЛЕДУЕМАЯ', 'ГИПОТЕЗА', 'СПОНСОР', 'ФИНАНСИРОВАНИЕ',
        'ДАННЫМ/ДОКУМЕНТАЦИИ', 'ЛИСТ', 'КАЧЕСТВА', 'ФАРМАКОКИНЕТИКИ', 'ЭТАП',
        'ИСКЛЮЧЕНИЕ', 'СПИСОК', 'ЗАПИСЕЙ', 'СИНОПСИС', 'СОДЕРЖАНИЕ',
        'ПРИЛОЖЕНИЯ', 'ЗАДАЧИ', 'КОНТРОЛЬ', 'ЦЕЛИ', 'ДОКУМЕНТЫ',
        'ИССЛЕДУЕМАЯ', 'РАБОТА', 'ИССЛЕДОВАТЕЛЬ', 'ОЦЕНКА', 'ПРИЕМ',
        'ЛЕКАРСТВЕННЫХ', 'ИСТОРИЯ', 'ПРЕПАРАТОВ', 'УЧАСТНИКОВ', 'ПРЯМОЙ',
        'БЕЗОПАСНОСТИ', 'ГЛАВНЫЙ', 'ПЕРВИЧНЫМ', 'СТАТИСТИКА', 'ДОКУМЕНТА',
        'СТРАХОВАНИЕ', 'НОРМАТИВНЫЕ', 'ПОДПИСНОЙ', 'ОБОСНОВАНИЕ', 'ДАННЫМИ',
        'ОБЕСПЕЧЕНИЕ', 'ГЛОССАРИЙ', 'ЭТИКА', 'ДОСТУП', 'СОКРАЩЕНИЙ', 'ДИЗАЙН',
        'ТЕРМИНОВ', 'ВЕДЕНИЕ', 'ПУБЛИКАЦИИ', 'ОТБОР', 'БИОЭКВИВАЛЕНТНОСТИ'
    }
lengths = [len(term) for term in exclude_terms]
(min(lengths), max(lengths))

(4, 19)

In [74]:
ABB_DICT_PATH = "data/abb_dict.csv"
abb_dict = pd.read_csv(ABB_DICT_PATH)

abbs = abb_dict['abbreviation'].values

In [76]:
lengths = [len(term) for term in abbs]
(min(lengths), max(lengths))

(1, 29)

In [92]:
[word for word in exclude_terms if len(word) <= 8 and word.isalpha()]

['ДИЗАЙН',
 'ГЛАВНЫЙ',
 'СПИСОК',
 'ПРЯМОЙ',
 'ПРИЕМ',
 'ПРОТОКОЛ',
 'КАЧЕСТВА',
 'ПЕРИОД',
 'ВЕДЕНИЕ',
 'ЭТАП',
 'ЭТИКА',
 'СИНОПСИС',
 'КОНТРОЛЬ',
 'ОТБОР',
 'ТЕРМИНОВ',
 'ИСТОРИЯ',
 'ОЦЕНКА',
 'СПОНСОР',
 'ЗАДАЧИ',
 'ЛИСТ',
 'РАБОТА',
 'ДОСТУП',
 'ЗАПИСЕЙ',
 'ЦЕЛИ',
 'ГИПОТЕЗА',
 'ДАННЫМИ']

In [87]:
[word for word in abbs if len(word) > 7 and word.isalpha()]

['AUCtotal']

### Test searching similar descriptions

In [305]:
import re

def normalize_text(text):
    """Normalize text by stripping, lowercasing, removing non-letter characters, and reducing multiple spaces."""
    
    print(f"Original: {text}")
    
    text = text.strip().lower()  # Strip spaces and lowercase
    print(f"Lowercased & Stripped: {text}")
    
    text = re.sub(r'[^a-zа-яё\s]', '', text)  # Keep only letters and spaces
    print(f"Non-letter Characters Removed: {text}")
    
    text = re.sub(r'\s+', ' ', text).strip()  # Reduce multiple spaces to single space
    print(f"Spaces Normalized: {text}")
    
    return text

# Test
normalize_text('Тиреотропный гормон (гипофиз) — важный компонент!')

Original: Тиреотропный гормон (гипофиз) — важный компонент!
Lowercased & Stripped: тиреотропный гормон (гипофиз) — важный компонент!
Non-letter Characters Removed: тиреотропный гормон гипофиз  важный компонент
Spaces Normalized: тиреотропный гормон гипофиз важный компонент


'тиреотропный гормон гипофиз важный компонент'

In [304]:
from process_doc import get_similar_description
ABB_DICT_PATH = "C:/Workspace/R-pharm/work/abbreviation_app/data/abb_dict.csv"
abb_dict = pd.read_csv(ABB_DICT_PATH)

custom_desc = 'тиреотропный - гормон (гипофиз)'
similar_pairs = get_similar_description(custom_desc, abb_dict, threshold=70)

if similar_pairs:
    print("\nSimilar pairs found:")
    for pair in similar_pairs:
        print(f"{pair['abbreviation']} - {pair['description']} (score: {pair['score']})")
else:
    print("\nNo similar pairs found.")


Similar pairs found:
ТТГ - Тиреотропный гормон (score: 78)


In [52]:
file_path = "C:/Workspace/R-pharm/work/abbreviation_app/data/abb_examples/TL-NLT-c-01_БИ_в2.0_19сен2018_драфт_ОФ.docx"
SECTION_PATTERN = 'СПИСОК СОКРАЩЕНИЙ' #'Список литературы'
NAMESPACE = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

In [53]:
def read_docx(file_path):
    # Load the document
    doc = Document(file_path)
    
    print("=== Paragraphs ===")
    for para in doc.paragraphs:
        if SECTION_PATTERN.casefold() in para.text.casefold():
            print(para.text)
    
    print("\n=== Tables ===")
    for i, table in enumerate(doc.tables):
        print(f"\nTable {i+1}:")
        for row in table.rows:
            print([cell.text for cell in row.cells])

In [54]:
read_docx(file_path)

=== Paragraphs ===
СПИСОК СОКРАЩЕНИЙ	5
СПИСОК СОКРАЩЕНИЙ

=== Tables ===

Table 1:
['\nБРОШЮРА ИССЛЕДОВАТЕЛЯ\n', '\nБРОШЮРА ИССЛЕДОВАТЕЛЯ\n']
['', '']
['Код продукта:', 'TL-NLT-c ']
['МНН:', 'нилотиниб']
['Торговое название', 'Нилотиниб-ТЛ']
['Лекарственная форма:', 'капсулы']
['Показание:\n', 'Положительный по филадельфийской хромосоме хронический миелоидный лейкоз (Ph+ ХМЛ))']
['Идентификационный номер протокола клинического исследования:', 'TL-NLT-c-01']
['Номер версии:', '2.0']
['Дата версии:', '19 сентября 2018 г.']
['Заменяет предыдущую версию номер:', 'Не применимо.']
['Дата предыдущей версии:', 'Не применимо.']
['Наименование/имя и адрес спонсора (монитора) клинического исследования:\n', 'ООО «Технология лекарств»\nЮридический и почтовый адрес: 141400, Московская обл., г. Химки, ул. Рабочая, д. 2a, стр. 31, пом. 21.\nТел.: +7 (495) 225-62-00, факс: +7 (495) 225-62-65.\nЭл. почта: info@drugsformulation.ru']
['Ф.И.О., должность, адрес и номер телефона назначенного спонсором медиц

In [77]:
file_name = "Список сокращений из БИ.docx"
folder_path = "C:/Workspace/R-pharm/work/abbreviation_app/data/abb_examples"
file_path = os.path.join(folder_path, file_name)
doc = Document(file_path)
first_element = doc.element.body[0]
first_element

<CT_Tbl '<w:tbl>' at 0x1ac08ba54a0>

In [85]:
tables = [block for block in doc.element.body if block.tag.endswith('tbl')]
len(tables)

1

In [78]:
len(doc.element.body) == 1 and doc.element.body[0].tag.endswith('tbl')

False

In [66]:
def debug_outline_level(file_path):
    doc = Document(file_path)
    print("=== Debug Outline Levels ===")
    
    for block in doc.element.body:
        if block.tag.endswith('p'):
            para_text = ''.join(
                node.text for node in block.findall('.//w:t', namespaces=NAMESPACE) if node.text
            )
            if SECTION_PATTERN.casefold() in para_text.casefold():
                print(f"\nParagraph: {para_text}")
                
                # Print the raw XML around the section
                para_xml = etree.tostring(block, pretty_print=True).decode()
                print("XML Structure:\n", para_xml)

In [67]:
debug_outline_level(file_path)

=== Debug Outline Levels ===

Paragraph: СПИСОК СОКРАЩЕНИЙ5
XML Structure:
 <w:p xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main" xmlns:w14="http://schemas.microsoft.com/office/word/2010/wordml" xmlns:wpc="http://schemas.microsoft.com/office/word/2010/wordprocessingCanvas" xmlns:cx="http://schemas.microsoft.com/office/drawing/2014/chartex" xmlns:cx1="http://schemas.microsoft.com/office/drawing/2015/9/8/chartex" xmlns:mc="http://schemas.openxmlformats.org/markup-compatibility/2006" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:r="http://schemas.openxmlformats.org/officeDocument/2006/relationships" xmlns:m="http://schemas.openxmlformats.org/officeDocument/2006/math" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:wp14="http://schemas.microsoft.com/office/word/2010/wordprocessingDrawing" xmlns:wp="http://schemas.openxmlformats.org/drawingml/2006/wordprocessingDrawing" xmlns:w10="urn:schemas-microsoft-com:office:word" xmlns:w15="http://schemas.microsoft.com/of

In [98]:
TMP_PATH = "C:/Workspace/R-pharm/work/abbreviation_app/data/docs_examples"
doc_file = [f for f in os.listdir(TMP_PATH) if f.endswith('.docx')]
if len(doc_file) != 1:
        raise FileNotFoundError("Expected exactly one .docx file in the folder.")
doc_path = os.path.join(TMP_PATH, doc_file[0])
print(f"[INFO] Processing document: {doc_path}")


[INFO] Processing document: C:/Workspace/R-pharm/work/abbreviation_app/data/docs_examples\RPH_001_CL01011070_Протокол_КИ_3_ф_в4_0_01дек2022_финал.docx


In [90]:
def extract_abb_table(doc, section_pattern=SECTION_PATTERN):
    found_section = False

    # If the entire doc is just one table, return it
    tables = [block for block in doc.element.body if block.tag.endswith('tbl')]
    if len(tables) == 1:
        print("[DEBUG] Document contains one table (ignoring paragraphs). Returning it.")
        return tables[0]

    for block in doc.element.body:
        if block.tag.endswith('p'):
            # Extract paragraph text
            para_text = ''.join(
                node.text for node in block.findall('.//w:t', namespaces=NAMESPACE) if node.text
            )

            # -- DEBUG PRINTS --
            if section_pattern.casefold() in para_text.casefold():
                print("")
                #print(f"\n[DEBUG] Found pattern in paragraph: '{para_text}'")

            # 1) Must contain SECTION_PATTERN
            if section_pattern.casefold() in para_text.casefold():

                # 2) Must NOT end with a digit (avoid "СПИСОК СОКРАЩЕНИЙ5", etc.)
                if para_text.strip().endswith(tuple("0123456789")):
                    #print("   [DEBUG] Skipped: ends with digit.")
                    continue

                # 3) Must NOT have a hyperlink (avoid ToC lines)
                link_node = block.find('.//w:hyperlink', namespaces=NAMESPACE)
                if link_node is not None:
                    #print("   [DEBUG] Skipped: hyperlink found (likely ToC).")
                    continue

                # 4) Must have some heading indication (pStyle or outlineLvl)
                para_style = block.find('.//w:pStyle', namespaces=NAMESPACE)
                outline_level = block.find('.//w:outlineLvl', namespaces=NAMESPACE)

                if para_style is not None or outline_level is not None:
                    #print("   [DEBUG] This paragraph triggers 'found_section'.")
                    found_section = True
                    continue
                else:
                    print("")
                    #print("   [DEBUG] Skipped: no heading style/outlineLvl.")

        # If we found the valid section, return the first table after it
        if found_section and block.tag.endswith('tbl'):
            #print("   [DEBUG] Table found after valid section heading.")
            return block

    return None

def parse_table(table_element):
    """
    Given an lxml table element, return a DataFrame. 
    Assumes the table has at least two columns: [abbreviation, description].
    """
    rows_data = []
    for row in table_element.findall('.//w:tr', namespaces=NAMESPACE):
        cell_texts = []
        for cell in row.findall('.//w:tc', namespaces=NAMESPACE):
            texts = cell.findall('.//w:t', namespaces=NAMESPACE)
            cell_text = ''.join(t.text for t in texts if t.text)
            cell_texts.append(cell_text.strip())
        rows_data.append(cell_texts)

    df = pd.DataFrame(rows_data)
    if df.shape[1] == 2:
        df.columns = ["abbreviation", "description"]
    return df

def process_folder(folder_path):
    """Process all .docx files in the given folder, printing debug info and table output."""
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.docx') and not file_name.startswith('~$'):
            file_path = os.path.join(folder_path, file_name)

            # 1) Print the file name first
            print(f"\n=== Processing: {file_name} ===")

            # 2) Extract the table (debug prints happen inside extract_abb_table)
            doc = Document(file_path)
            table_element = extract_abb_table(doc)

            # 3) If a table is found, parse and print it; otherwise say "No table found."
            if table_element is not None:
                df = parse_table(table_element)
                print("\nFirst Rows of the Table:")
                print(df.head())  # Output first rows
            else:
                print("\nNo table found.")


In [92]:
FOLDER_PATH = "C:/Workspace/R-pharm/work/abbreviation_app/data/abb_examples"
SECTION_PATTERN = "СПИСОК СОКРАЩЕНИЙ"

process_folder(FOLDER_PATH)


=== Processing: CL01909121_DT-LPT_БИ_в1.0_19авг2022_финал.docx ===



First Rows of the Table:
  abbreviation                                        description
0          AUC  площадь под фармакокинетической кривой «концен...
1         BCRP  breast cancer resistance protein/белок устойчи...
2         Cmax  максимальная концентрация лекарственного вещес...
3          CYP                                           цитохром
4           CV                               коэффициент вариации

=== Processing: DT-ABM_CL011138276_БИ_в.1.0_28авг2024_финал.docx ===



First Rows of the Table:
  abbreviation                                        description
0          AUC  Площадь под фармакокинетической кривой «концен...
1         Cl/F                                      Общий клиренс
2         Cmax  Максимальная концентрация лекарственного вещес...
3    CDK4/CDK6                      Циклин-зависимые киназы 4 и 6
4           CV                               Коэффициент вариации

=== Processin